In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showerror, showwarning
import sqlite3


class People(tk.Tk):
    '''Класс для внесения информации о людях в базу данных'''
    
    db_name = 'people.db'
    '''Атрибут класса с наименованием файла базы данных(бд)'''
    
    def __init__(self):
        '''Устанавливает все необходимые атрибуты для объекта, содержит в себе параметры:
        - self.geometry() - метод, предустановленное значение, принимающие строку с размером экрана пользователя;
        - self.title() - метод, предустановленное значение, принимающие строку с наименованием экрана пользователя;
        - self.resizable() - метод, предустановленное значение, принимающие 2 булева или аналога их чисел для отключения 
        изменения размера экрана пользователя;
        - в style конфигураторе задается стилистика;
        - виджеты для отображения фрейма с отображением выбранной и удаленной записи из таблицы people;
        - виждеты отображение записей, колонок и наименований, вертикальной прокрутки таблицы people;
        - виджеты для выбора дальнейших действий с записями из таблицы people, добавить/удалить/завершить работу, 
        при выборы завершения вызывается метод self.closing();
        - список для хранения данных для отображения выбранную запись или запись при удалении;
        - метод self.db_create() для создания бд people.db и таблицы people, если они не существуют, 
        и наполнения даными таблицы при первом запуске, если она не существовала, определенными словарем в методе;
        - метод self.get_words() на наполнения таблицей данными;
        - метод для интерактивности отображения выбранного значения.'''
        super().__init__()
        self.geometry('700x400')
        self.title('Список людей')
        self.resizable(0,0)
        style = ttk.Style()
        style.configure(".", font="helvetica 11", foreground="#004D40", background="#B2DFDB")

        # фрейм для отображение выбранной и удаленной записи из таблицы people
        self.frame_top = tk.LabelFrame(self, text='', foreground="#004D40", font="helvetica 13")
        self.frame_top.grid(row=0, column=0, pady=20, padx=20, sticky=tk.W+tk.E)
        self.label = ttk.Label(self.frame_top, text='', font="helvetica 13")
        self.label.pack(side=tk.TOP,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)

        # настройка отображение записей, колонок и вертикальной прокрутки таблицы people
        columns = ("first_name", "last_name", "age")
        self.tree = ttk.Treeview(self, show="headings", columns=columns)
        self.tree.column('first_name', width=220, anchor=tk.NW)
        self.tree.column('last_name', width=220, anchor=tk.NW)
        self.tree.column('age', width=220, anchor=tk.NW)
        self.tree.heading("first_name", text="Имя", anchor=tk.CENTER)
        self.tree.heading("last_name", text="Фамилия", anchor=tk.CENTER)
        self.tree.heading("age", text="Возраст", anchor=tk.CENTER)
        ysb = ttk.Scrollbar(self, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=ysb.set)
        self.tree.grid(row=1, column=0)
        ysb.grid(row=1, column=1, sticky=tk.N + tk.S)
        self.rowconfigure(1, weight=1)
        self.columnconfigure(0, weight=1)
    
    

        # кнопки выбора дальнейших действий с таблицей people
        frame = tk.LabelFrame(self, text='Выберите действие с таблицей people:', foreground="#004D40", font="helvetica 13")
        frame.grid(row=6, column=0, pady=20, padx=20, sticky=tk.W+tk.E)
        self.button2=ttk.Button(frame, text='Добавить новую запись', command=self.add_word)
        self.button2.pack(side=tk.LEFT,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.button1=ttk.Button(frame, text='Удалить выбранную запись', command=self.delete_word)
        self.button1.pack(side=tk.LEFT,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.button3 = ttk.Button(frame, text='Завершить работу', command=lambda: self.closing())
        self.button3.pack(side=tk.LEFT,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)

        # список для хранения данных для отображения выбранную запись или запись при удалении
        self.lst = []

        # создание и первичное наполнение бд предустановленными данными, если ее нет
        self.db_create()
        
        # заполнение таблицы
        self.get_words()

        # связывание интерактивности выбора записей
        self.tree.bind("<<TreeviewSelect>>", self.item_selected)

    
    def run_query(self, query, parameters=()):
        '''метод, обеспечивающий запрос и подключение к базе данных people, принимающий на вход sql-запрос и параметры запроса,
        возвращает результат запроса.'''
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            result = cursor.execute(query, parameters)
            conn.commit()
        return result

    def db_create(self):
        '''метод обеспечивающий создание базы данных people.db, если файла базы данных нет в данной директории, создание таблицы people, 
        если ее не существует и обеспечивает наполнение таблицы people данными из заранее определенного словаря в методе,
        при создании таблицы при первом запуске, если таблица не существовала.'''
        conn = sqlite3.connect(self.db_name)
        cur = conn.cursor()
        data = cur.execute("select count(*) from sqlite_master where type ='table' and name ='people'")
        for row in data:
            if row[0] == 0:
                cur.execute("""CREATE TABLE IF NOT EXISTS people(
                id integer primary key,
                first_name text,
                last_name text,
                age integer);""")
                sql = 'INSERT INTO people(first_name, last_name, age) values(?, ?, ?)'
                initial_data = [{"first_name": "Иван", "last_name": "Иванов", "age": 30},
                                {"first_name": "Мария", "last_name": "Петрова", "age": 25},
                                {"first_name": "Алексей", "last_name": "Сидоров", "age": 40}]
                data = []
                for i in initial_data:
                    data.append(tuple(i.values()))
                cur.executemany(sql, data)
            conn.commit()
        
            
    def get_words(self):
        '''метод обеспечивающий подключение к базе данных и получение из нее данных по sql-запросу, селектом'''
        with sqlite3.connect(self.db_name) as conn:
            cur = conn.cursor()
            cur.execute('''SELECT first_name, last_name, age, id FROM people''')
        db_rows = cur.fetchall()
        [self.tree.delete(i) for i in self.tree.get_children()]
        [self.tree.insert('', 'end', values=row) for row in db_rows]


    
    def add_word(self):
        '''метод для создания виждетов дополнительного окна при добавлении записи для внесения пользователем имени, фамилии и возраста.
        Имеет кнопки:
        - записать, вызывающий метод self.saving_edit() для внесения данных в таблицу бд, 
        в который передаются данные, полученные из полей для внесения имени, фамилии и возраста;
        - закрыть self.edit_wind.destroy(), если пользователь передумал записывать данные, которая закрывает окно.
        Содержит в себе:
        self.edit_wind.geometry() - метод, предустановленное значение, принимающие строку с размером экрана пользователя;
        self.edit_wind.title() - метод, предустановленное значение, принимающие строку с наименованием экрана пользователя;
        self.edit_wind.resizable() - метод, предустановленное значение, принимающие 2 булева или аналога их чисел для отключения 
        изменения размера экрана пользователя.'''
        
        self.edit_wind = tk.Toplevel(self)
        self.edit_wind.geometry("400x300")
        self.edit_wind.title('Добавить запись')
        self.edit_wind.resizable(0,0)

        # фрейм для внесения имени пользователем
        self.frame_name = tk.LabelFrame(self.edit_wind, text = "Введите имя:", foreground="#004D40", font="helvetica 12")
        self.frame_name.pack(side=tk.TOP, anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.entry_name = ttk.Entry(self.frame_name, font="helvetica 13")
        self.entry_name.pack(side=tk.TOP, anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)

        # фрейм для внесения фамилии пользователем
        self.frame_last_name = tk.LabelFrame(self.edit_wind, text = "Введите фамилию:", foreground="#004D40", font="helvetica 12")
        self.frame_last_name.pack(side=tk.TOP, anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.entry_last_name = ttk.Entry(self.frame_last_name, font="helvetica 13")
        self.entry_last_name.pack(side=tk.TOP, anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)

        # фрейм для внесения возраста пользователем
        self.frame_age = tk.LabelFrame(self.edit_wind, text = "Введите возраст:", foreground="#004D40", font="helvetica 12")
        self.frame_age.pack(side=tk.TOP, anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.entry_age = ttk.Entry(self.frame_age,font="helvetica 13")
        self.entry_age.pack(side=tk.TOP, anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)

        # фрейм для выбора пользователем дальнейших действий с внесенными данными
        self.frame_buttom = tk.LabelFrame(self.edit_wind)
        self.frame_buttom.pack(side=tk.TOP,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.save_button=ttk.Button(self.frame_buttom, text="Записать", 
        command=lambda: self.saving_edit(self.entry_name.get(), self.entry_last_name.get(), self.entry_age.get()))
        self.save_button.pack(side=tk.LEFT,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)
        self.close_button=ttk.Button(self.frame_buttom, text="Закрыть", command=lambda: self.edit_wind.destroy())
        self.close_button.pack(side=tk.LEFT,anchor=tk.NW, fill=tk.X, expand=True, padx=5, pady=5)

    def saving_edit(self, first_name, last_name, age):
        '''метод для сохранения внесения изменений в таблице people бд, вызывом метода self.run_query() 
        и метода self.get_words() для отображения актуализированных данных из таблицы бд.
        На вход принимает значения переменных имени, фамилии и возраста, очищает их от лишних пробелов с начала и конца строки
        и приводит переменные first_name, last_name к первой заглавной букве каждого слова переменных.
        Проверяет если есть пустые значении в переменных, используя len(), при наличии выдает предупреждение, закрывает окно и открывает новое,
        вызывая метод self.add_word(),заполняя значения по не пустым переменным.
        При отсутствии пустых значений проверяет является введенный возраст является ли целым числом, методом isdigit(), если нет выдает ошибку и 
        закрывает окно, открывая новое, вызывая метод self.add_word(),заполняя значения внесенных переменных,
        если является вызывает метод self.run_query(), передавая sql-запрос на внесение информации в таблице бд и параметры, 
        закрывает дополнительное окно и отображает актуализированные данные в таблицу.'''

        # очищение данных внесенных пользователем
        first_name = first_name.strip().title()
        last_name = last_name.strip().title()
        age = age.strip()

        # проверка на пустые значения данных
        if len(first_name) == 0 or len(last_name) == 0 or len(age) == 0:
            showwarning(title = "Warning", message = "Не указано имя и/или фамилия и/или возраст")
            self.edit_wind.destroy()
            self.add_word()
            self.entry_name.insert(1, first_name)
            self.entry_last_name.insert(1, last_name)
            self.entry_age.insert(1, age)
        else:
            # проверка является ли возраст целым числом
            if age.isdigit() == False:
                showerror(title="Error", message = "Возраст указан не целым числом.")
                self.edit_wind.destroy()
                self.add_word()
                self.entry_name.insert(1, first_name)
                self.entry_last_name.insert(1, last_name) 
            else:
                # заполнение таблицы бд и отображение данных
                query = 'INSERT INTO people VALUES(NULL, ?, ?, ?)'
                parameters = (first_name, last_name, age)
                self.run_query(query, parameters)
                self.edit_wind.destroy()
                self.get_words()
        
 
    
    def delete_word(self):
        '''метод удаление выбранного пользователем записи из таблицы people бд. Метод обращается к переменной self.lst: 
        - в случае если длина списка больше нуля, меняет значение self.frame_top["text"] и self.label["text"], 
        запускает метод self.run_query(), передавая sql-запрос на удаление, а также id номер записи на удаление из таблицы бд,
        отображает данные пользователю об уданенной записи, обнуляет переменную self.lst, актуализирует список таблицы бд,
        вызывая метод self.get_words(),
        - в обратном случае выдает предупреждение, что запись для удаление не выбрана.'''
        if len(self.lst) > 0:
            self.frame_top["text"] = ""
            number = self.lst[3] 
            query = 'DELETE FROM people WHERE id = ?'
            self.run_query(query, (number,))
            self.label["text"] = f'Запись c id: {self.lst[3]} - "{self.lst[0]} {self.lst[1]} {self.lst[2]}" удалена.'
            self.lst=[]
            self.get_words()
        else:
            self.label['text'] = 'Выберите запись, которую нужно удалить'

    def item_selected(self, event):
        '''метод для работы с данными, получаемых из self.tree.selection(), которые видет пользователь при выборе записи.
        Метод изменяет значение self.frame_top["text"] и в случае если значение не выбрано пользователем, проверяется с помощью 
        вызова len() изменяет значение self.label["text"], заполняя self.lst выбранными значения пользователем'''
        selected_people =""
        for selected_item in self.tree.selection():
            item = self.tree.item(selected_item)
            person = item["values"]
            selected_people =f"{selected_people}{person}"
            self.frame_top["text"] = "Выбрана запись из таблицы people:"
        if len(selected_people) > 0:
            self.label["text"] = f'{person[0]} {person[1]} {person[2]} - id: {person[3]}'
            self.lst = [person[0], person[1], person[2], person[3]]
    
    def closing(self):
        '''метод для закрытия бд и закрытия окна'''
        conn = sqlite3.connect(self.db_name)
        cur = conn.cursor()
        conn.close()
        self.destroy()
            
# запуск программы и обработка исключений
if __name__ == '__main__':
    try:
        application = People()
        application.mainloop()
    except Exception as e:
        print(f"Произошла непредвиденная ошибка {e}. Запустите программу заново.")
    